In [1]:
import math
import torch
from torch import empty , cat , arange
from torch . nn . functional import fold , unfold

#to delete
from torch import nn
from others.modules import *


In [2]:
# Extract images

noisy_imgs_1 , noisy_imgs_2 = torch.load('data/train_data.pkl')
noisy_imgs , clean_imgs = torch.load ('data/val_data.pkl')

noisy_imgs = noisy_imgs/255
clean_imgs = clean_imgs/255

# select a preset of images:

imgs_1 = noisy_imgs_1[:10000]/255
imgs_2 = noisy_imgs_2[:10000]/255
print(imgs_1.shape)

torch.Size([10000, 3, 32, 32])


In [5]:
class Model(object):
    def __init__(self):
        self.sequence = Sequential(Convolution(3, 32, kernel_size = 4, padding = 1, dilation = 0, stride = 2),
                                   ReLU(),
                                   Convolution(32,32, kernel_size = 4, padding = 1,dilation = 0,  stride = 2),
                                   ReLU(),
                                   ConvolutionTransposed(32,32, kernel_size = 4,padding = 1, dilation = 0, stride = 2),
                                   #Upsample(2),
                                   #Convolution(20,20, kernel_size = 3, stride = 1),
                                   ReLU(),
                                   ConvolutionTransposed(32,3, kernel_size = 4,padding = 1, dilation = 0, stride = 2),
                                   #Upsample(2),
                                   #Upsample(3),
                                   #Convolution(20,3, kernel_size = 5, stride = 1),
                                   Sigmoid())
                                   

        self.criterion = MSE()
        self.optimizer = SGD(self.sequence, lr = 1e-4)
        
        self.nb_samples = 10000
        self.epochs = 10
        print('Model Initialisation')

    def train(self,train_input,train_target):
    
        for epoch in range(self.epochs):

            if not epoch % 10: 
                print('*******************************************************************************',epoch)
                print('epoch',epoch)

            for n in range(self.nb_samples):

                if not n % 10000: 
                    print('sample: ', n)
                    print('weight',self.sequence.layers[0].weight[0,0,0])
                    print('grad',self.sequence.layers[0].grad[0,0,0])
                    #print('loss', loss)
                #print('img', train_input[n:n+1].shape)

                predict_img = self.sequence.forward(train_input[n:n+1])
                #print('optimizer zero grad')
                self.optimizer.zero_grad()
                #print('prediction', predict_img.shape)
                loss = self.criterion.forward(predict_img, train_target[n:n+1])
                #print('loss backward', self.criterion.backward())
                backward_img = self.sequence.backward(self.criterion.backward())
                if not n % 10000:
                    print('loss', loss)
                    #print('grad conv0',self.sequence.layers[0].grad[0,0,0])
                    #print('grad sigmoid',self.sequence.layers[8].grad[0,0,0,0])
                    #print('grad upsample',self.sequence.layers[7].grad[0,0,0,0])
                    #print('grad relu',self.sequence.layers[6].grad[0,0,0,0])
                    #print('*********************************************************************************')
                self.optimizer.step()
        

        return backward_img
    
    def predict(self,test_input):
        prediction = torch.empty(test_input.shape)
            
        for n in range(test_input.shape[0]):
            prediction[n:n+1] = self.sequence.forward(test_input[n:n+1])

        return prediction
        

In [6]:
#img = torch.randn(1, 3, 32, 32)
#img_error = torch.randn(1, 3, 32, 32)

model = Model()

output = model.train(imgs_1, imgs_2)



print('out img', output.shape)



Model Initialisation
******************************************************************************* 0
epoch 0
sample:  0
weight tensor([ 0.2804,  0.1690, -0.8568,  1.3221])
grad tensor([0.0000e+00, 7.1767e+22, 7.0295e+28, 6.1949e-04])
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
loss tensor(0.3053)
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32,

img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch

img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])


Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 

Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])

...

img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])


img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch

img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch

Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 8, 8])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 3, 32, 32])
Sequential backward
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 32, 16, 16])
img torch.Size([1, 

KeyboardInterrupt: 

In [71]:
def psnr ( denoised , ground_truth ) :
        # Peak Signal to Noise Ratio : denoised and ground_truth have range [0 , 1]
        mse = torch.mean (( denoised - ground_truth ) ** 2)
        return -10 * torch.log10 ( mse + 10** -8)

In [72]:
print(noisy_imgs.shape[0])

prediction = model.predict(noisy_imgs)


print(prediction.shape)

1000
torch.Size([1000, 3, 32, 32])


In [73]:
img_denoise_psnr = psnr(prediction ,clean_imgs )

In [74]:
print("psnr", img_denoise_psnr)

psnr tensor(5.9195)


In [36]:
img_denoise_psnr = psnr(noisy_imgs ,clean_imgs )
print("psnr", img_denoise_psnr)


psnr tensor(19.6150)
